# Risk Transfer Formula

Current Todo List:
    * read from sas files
    * integrate PLRS from algorex
    * figure out how to use GCF

## 0. Introduction and Setup
The goal of the risk transfer formula (described here: https://www.cms.gov/mmrr/Downloads/MMRR2014_004_03_a04.pdf) is to take the liability risk, allowable premium rating, actuarial value, induced demand, geographic cost, and statewide average premium to calculate transfers among plans.

In [1]:
import math
import numpy as np 
import pandas as pd

data = pd.read_csv("data/fakedata-1.csv")

## 1. Formula
The transfer formula averages all risk scores in a plan, makes certain adjustments, and then calculates the funds transfered between plans. In the formula, _i_ represents a plan (i.e. bronze, gold, silver, platinum, etc.), of which there are N, numbered 1, 2, ... N. 

The formula is:
![The Risk Transfer Formula](images/TransferFormula.PNG)

## 2. Definitions of Factors within the Risk Transfer Formula
First, I define five plans: bronze, silver, gold, platinum, and catastrophic. Next, I alphabetically define ARF, AV, GCF, IDF, PLRS, and S, the components to be used in the calculation.

In [2]:
plans = ["bronze", "silver", "gold", "platinum", "catastrophic"]
ARF = dict()
AV = dict()
GCF = dict()
IDF = dict()
PLRS = dict()
S = dict()

### 2a. Allowable Rating Factor (ARF)
ARF reflects the relative premium that plans are permitted to charge, given the allowable rating factors of its enrollees. ARF only adjusts for age rating, not any other factor. Age rating is generally on the 3:1 ratio federal age rating curve, although in different states this could be 2:1 or 2.5:1. 

Todo: look into auto calculating the values so they will not have to be manually inputted through a csv.

In [6]:
# define ARF - age rating adjustment
class Federal_Age_Rating_Curve:
    def __init__(self): 
        pass
    
    def set_age_curve_values(self):
        self.scores = pd.read_csv("federal_age_rating_curve_3-1.csv")
    
    def generate_one_score(self, age):
        if age <= 24:
            return 1.0
        elif age >= 64:
            return 3.0
        else:
            score = self.scores.where(self.scores["Age"] == age).dropna()
            return score["Premium_Ratio"].get_value(score.first_valid_index())
    
    def generate_average_score(self, ages):
        # faster way: count how many of each age there is in ages and then multiply by cost and add
        sum_scores = 0
        count = 0
        for age in ages:
            sum_scores = sum_scores + self.generate_one_score(age)
            count = count + 1
        if count == 0:
            return 0
        return sum_scores / count

f = Federal_Age_Rating_Curve()
f.set_age_curve_values()

for plan in plans:
    this_plan_data = data.where(data["Plan"] == plan).dropna()
    this_plan_ages = this_plan_data["Age"]
    ARF[plan] = f.generate_average_score(this_plan_ages)

print("ARF values: " + str(ARF))

ARF values: {'bronze': 1.8258516746411475, 'silver': 1.8164230769230769, 'gold': 1.8631691542288544, 'platinum': 1.7880721153846155, 'catastrophic': 1.7458799999999997}


### 2b. Actuarial Value (AV)
AV is associated with each plan's metal level. AV measures the proportion of enrollees' total medical expenditures paid by the plan. The values here are copied from Exhibit 1 on page E7 of the Risk Transfer Formula paper.

In [8]:
# define AV
AV["bronze"] = 0.6
AV["silver"] = 0.7
AV["gold"] = 0.8
AV["platinum"] = 0.9
AV["catastrophic"] = 0.57

print("AV values: " + str(AV))

AV values: {'bronze': 0.6, 'silver': 0.7, 'gold': 0.8, 'platinum': 0.9, 'catastrophic': 0.57}


According to this article: https://www.cms.gov/CCIIO/Resources/Regulations-and-Guidance/Downloads/Revised-Final-2018-AV-Methodology-41317.pdf these values can change by -0.04 +0.02, except bronze, when can be up to -0.04 +0.05. The statute groups health plans into four tiers: bronze, with an AV of 60 percent; silver, with an AV of 70 percent; gold,
with an AV of 80 percent; and platinum, with an AV of 90 percent. Here are the steps for calculating the 2018 AV:
- Step 1: Set the metal tier (by identifying the continuance tables on which the calculation will be based)
- Step 2: Calculate average expenses over all enrollees (by identifying the denominator of the AV calculation, the average cost over all enrollees for a plan of the specified metal level)
- Step 3: Calculate expenses covered by employer contributions to HSAs and HRAs, if applicable
- Step 4: Calculate plan-covered expenses for spending before the deductible is met
- Step 5: Determine applicable enrollee spending level for MOOP
- Step 6: Calculate plan-covered expenses for spending between the deductible and the MOOP (in the coinsurance range)
- Step 7: Calculate plan-covered expenses for spending above the MOOP
- Step 8: Apply tiered network, if applicable (to calculate AV in Tier 2)
- Step 9: Calculate AV and corresponding metal tier (to assign AV and metal tier)

Other necessary background information includes necessary deductibles, coinsurance, and MOOPs consistent with the choice of integrated or separate deductibles and MOOPs for medical and drug expenses.

Helpful for me: what is MOOP? Maximum out of pocket payment.

### 2c. Geographic Cost Factor (GCF)
GCF reflects the medical cost structure in the geographic location of the plan's enrollees. "The enrollment-weighted statewide average of plan GCF values equals 1.0, so deviations from 1.0 can be interpreted as the percentage by which any geographic area’s costs deviate from the state average. In other words, a GCF equal to 1.15 indicates that the plan operates in a geographic area where costs are, on average, 15 percent higher than the statewide average" (Page E9).

The GCF is calculated based on the observed average silver plan premiums in a geographic area relative to the state-wide average silver plan premium.

N.B. Since Oscar only operates in one area, I currently simplified and set all of these values to 1.0. GCF should be a weighted average of all of the states that the patients are from.

In [17]:
# define GCF
GCF["bronze"] = 1.0
GCF["silver"] = 1.0
GCF["gold"] = 1.0
GCF["platinum"] = 1.0
GCF["catastrophic"] = 1.0

### 2d. Induced Demand Factor (IDF)
IDF reflects the anticipated induced demand associated with the plan's cost sharing (metal) level. The values here are copied from Exhibit 2 on page E7 of the Risk Transfer Formula paper.

In [16]:
# define IDF
IDF["bronze"] = 1.0
IDF["silver"] = 1.03
IDF["gold"] = 1.08
IDF["platinum"] = 1.15
IDF["catastrophic"] = 1.0

### 2e. Plan Liability Risk Score (PLRS)
PLRS reflects the plan’s actuarial value as well as the plan’s enrollee health status risk (including health risk due to age). PLRS is calculated in the Risk Model Python notebook, adapted from Algorex Health's Github.

Temp values are 1.0.

https://github.com/guanzgrace/hcc-python

In [13]:
# define PLRS 
PLRS["bronze"] = 1.0
PLRS["silver"] = 1.0
PLRS["gold"] = 1.0
PLRS["platinum"] = 1.0
PLRS["catastrophic"] = 1.0

### 2f. Share of Marketwide Enrollment (S)
S reflects the share of marketwide enrollement each plan has.

In [10]:
# define S

for plan in plans:
    this_plan_data = data.where(data["Plan"] == plan).dropna()
    S[plan] = this_plan_data.size / data.size

print("S values: " + str(S))

S values: {'bronze': 0.20899999999999999, 'silver': 0.182, 'gold': 0.20100000000000001, 'platinum': 0.20799999999999999, 'catastrophic': 0.20000000000000001}


### 2g. Other potentially useful definitions
P_bar = average revenue

PL_bar = average liability

## 3. Implementation of the Risk Transfer Formula

In [19]:
P_bar = 1.0
T = dict()

first_numerator = dict()
second_numerator = dict()
first_denominator = 0
second_denominator = 0

for plan in plans:
    first_numerator[plan] = PLRS[plan] * IDF[plan] * GCF[plan]
    second_numerator[plan] = AV[plan] * ARF[plan] * IDF[plan] * GCF[plan]
    first_denominator += S[plan] * first_numerator[plan]
    second_denominator += S[plan] * second_numerator[plan]
    
for plan in plans:
    T[plan] = ((first_numerator[plan] / first_denominator) +
        (second_numerator[plan] / second_denominator)) * P_bar
    
print("T: " + str(T))

T: {'bronze': 1.7467253116802404, 'silver': 1.9309704863070949, 'gold': 2.1967712046515766, 'platinum': 2.4384665794818261, 'catastrophic': 1.6737286034187595}
